In [6]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, precision_score, recall_score

cancer = load_breast_cancer()
features = cancer.feature_names
X = cancer.data
y = cancer.target
print(X.shape)

X = pd.DataFrame(X,columns=features)
y = pd.Series(y,index=X.index)

# split data into training and testing sets
rs = ShuffleSplit(test_size=0.3, n_splits=1, random_state=0)
for train_idx,test_idx in rs.split(X):  # this looks silly
    pass

train_split = round(len(train_idx) / 2)
train1_idx = train_idx[:train_split]
train2_idx = train_idx[train_split:]
X_train = X.loc[train_idx]
X_train_1 = X.loc[train1_idx]
X_train_2 = X.loc[train2_idx]
X_test = X.loc[test_idx]
y_train = y.loc[train_idx]
y_train_1 = y.loc[train1_idx]
y_train_2 = y.loc[train2_idx]
y_test = y.loc[test_idx]

xg_train_0 = xgb.DMatrix(X_train, label=y_train)
xg_train_1 = xgb.DMatrix(X_train_1, label=y_train_1)
xg_train_2 = xgb.DMatrix(X_train_2, label=y_train_2)
xg_test = xgb.DMatrix(X_test, label=y_test)

num_round = 5
verbose_eval = 5
watch_list = [(xg_test, 'xg_test')]

params = {'objective': 'binary:logistic', 'verbose': False, 'eval_metric': ['auc']}
print('full train\t'); 
model_0 = xgb.train(params, xg_train_0, 10, watch_list, verbose_eval=verbose_eval)
print('model 1 \t'); 
model_1 = xgb.train(params, xg_train_1, num_round, watch_list, verbose_eval=verbose_eval)
model_1.save_model('model_1.model')
print('model 2 \t'); 
model_2_v1 = xgb.train(params, xg_train_2, num_round, watch_list, verbose_eval=verbose_eval)
print('model 1+2\t, this logs show continue train, but got a test score same to model 1?'); 
model_2_v2 = xgb.train(params, xg_train_2, num_round, watch_list, verbose_eval=verbose_eval, xgb_model=model_1)

params.update({
    'process_type': 'update',
    'updater': 'refresh',
    'refresh_leaf': True,
})
print('model 1+update2\t, this logs either do not show continue train, nor do not get a better score?'); model_2_v2_update = xgb.train(params, xg_train_2, num_round, watch_list, verbose_eval=verbose_eval, xgb_model=model_1)


print(model_0.predict(xg_test).shape)

def validation(model, x_data, y_data):
    y_data_proba = model.predict(xg_test)
    y_data_pred = (y_data_proba>=0.5).astype(int)
    
    acc = accuracy_score(y_test, y_data_pred)
    prec = precision_score(y_test, y_data_pred)
    recall = recall_score(y_test, y_data_pred)
    roc = roc_auc_score(y_test, y_data_proba)
    return acc, prec, recall, roc

print('full train\t acc: %.3f, prec: %.3f, recall: %.3f, roc: %.3f' % validation(model_0, xg_test, y_test)) # benchmark
print('model 1 \t acc: %.3f, prec: %.3f, recall: %.3f, roc: %.3f' % validation(model_1, xg_test, y_test))  
print('model 2 \t acc: %.3f, prec: %.3f, recall: %.3f, roc: %.3f' % validation(model_2_v1, xg_test, y_test))  # "before"
print('model 1+2\t acc: %.3f, prec: %.3f, recall: %.3f, roc: %.3f' % validation(model_2_v2, xg_test, y_test))  # "after"
print('model 1+update2\t acc: %.3f, prec: %.3f, recall: %.3f, roc: %.3f' % validation(model_2_v2_update, xg_test, y_test))  # "after"

(569, 30)
full train	
[0]	xg_test-auc:0.972443
[5]	xg_test-auc:0.993974
[9]	xg_test-auc:0.996546
model 1 	
[0]	xg_test-auc:0.919165
[4]	xg_test-auc:0.971781
model 2 	
[0]	xg_test-auc:0.944591
[4]	xg_test-auc:0.991329
model 1+2	, this logs show continue train, but got a test score same to model 1?
[0]	xg_test-auc:0.980012
[4]	xg_test-auc:0.99177
model 1+update2	, this logs either do not show continue train, nor do not get a better score?
[0]	xg_test-auc:0.919165
[4]	xg_test-auc:0.970606
(171,)
full train	 acc: 0.971, prec: 0.981, recall: 0.972, roc: 0.997
model 1 	 acc: 0.912, prec: 0.935, recall: 0.926, roc: 0.972
model 2 	 acc: 0.930, prec: 0.921, recall: 0.972, roc: 0.991
model 1+2	 acc: 0.959, prec: 0.955, recall: 0.981, roc: 0.992
model 1+update2	 acc: 0.906, prec: 0.934, recall: 0.917, roc: 0.971
